In [29]:
import pandas as pd
import googlemaps
import os
import numpy as np
#Get API Key from TXT file -- in gitignore
apikey = 'apikey.txt'



In [31]:
#Return contents of a file
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)
api_key = get_file_contents(apikey)

In [32]:
#Run Google Maps
gmaps = googlemaps.Client(key=api_key)

In [33]:
#Import files

#First NEA files, of which there are multiple
files = os.listdir('NEA Grants')
the_list = []


for f in files:
    print(f)
    data = pd.read_excel('NEA Grants/' + f)
    the_list.append(data)

NEA = pd.concat(the_list)
NEH = pd.read_excel('NEHAwardSearchResults.xlsx')

2021.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2020.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2024.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2023.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2022.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [34]:
#Delete Rows with nan values
NEA = NEA.dropna(how='all', inplace=False)
NEH = NEH.dropna(how='all', inplace=False)

In [35]:
#Delete Cognressional District col so it's not confused with later imports
del NEA['Congressional District']

In [36]:
#Exporting Cleaned Full Grant list
NEA.to_csv("NEA_Full_Grant.csv")
NEH.to_csv("NEH_Full_Grant.csv")

In [37]:
#Get only columns only needed for Lat/Long Look-Up
NEA_Orgs = NEA[['Organization Name','City', 'State','Zip']]
NEH_Orgs = NEH[['Organization', 'Organization City', 'Organization State',
       'Organization Postal Code',]]

In [38]:
#Drop Duplicates
NEA_Orgs = NEA_Orgs.drop_duplicates()
NEH_Orgs = NEH_Orgs.drop_duplicates()

In [39]:
#Create full address column for lookup
NEA_Orgs['Full Address'] = NEA_Orgs['Organization Name'] + ", " + NEA_Orgs['City'] + ", " + NEA_Orgs['State'] + " " +str(NEA_Orgs['Zip'])
NEH_Orgs['Full Address'] = NEH_Orgs['Organization'] + ", " + NEH_Orgs['Organization City'] + ", " + NEH_Orgs['Organization State'] + " " + str(NEH_Orgs['Organization Postal Code'])

In [40]:
#Check size of NEA_Orgs
NEA_Orgs.shape

(5751, 5)

In [41]:
#Check size of NEH_Orgs
NEH_Orgs.shape

(1745, 5)

In [42]:
#Split Dataframes in manageable chunks to not exceed api limits
NEA_Orgs1, NEA_Orgs2, NEA_Orgs3, NEA_Orgs4, NEA_Orgs5 = np.array_split(NEA_Orgs, 5)
NEH_Orgs1, NEH_Orgs2 = np.array_split(NEH_Orgs, 2)

NEA_List = [NEA_Orgs1, NEA_Orgs2, NEA_Orgs3, NEA_Orgs4, NEA_Orgs5]
NEH_List = [NEH_Orgs1, NEH_Orgs2]


In [43]:
#Function to run geocoding
def geocoding(df):
    text_list = df["Full Address"].to_list()
    Lat_List = []
    Long_List = []
    for address in text_list:
        geo = gmaps.geocode(address)
        try: 
            lat = geo[0]['geometry']['bounds']['northeast']['lat']   
            lng = geo[0]['geometry']['bounds']['northeast']['lng']

        except:
            try: 
                lat = geo[0]['geometry']['location']['lat']   
                lng = geo[0]['geometry']['location']['lng'] 
            except:
                lat = "none"
                lng = "none"
        Lat_List.append(lat)
        Long_List.append(lng)
    df["Latitude"] = Lat_List
    df["Longitude"] = Long_List
    return df

In [49]:
#Running Geocoding and saving files
#Commenting out to avoid accidental runs NEH_List[0] = geocoding(NEH_List[0])
NEH_List[0]

,Organization,Organization City,Organization State,Organization Postal Code,Full Address,Latitude,Longitude
0,Tufts University,Somerville,MA,02144-2401,"Tufts University, Somerville, MA 0 02144...",42.418118,-71.072995
1,Northeastern Oklahoma State University,Tahlequah,OK,74464-2301,"Northeastern Oklahoma State University, Tahleq...",35.952788,-94.941901
2,Princeton University,Princeton,NJ,08540-5228,"Princeton University, Princeton, NJ 0 02...",40.391755,-74.617508
3,Goucher College,Baltimore,MD,21204-2753,"Goucher College, Baltimore, MD 0 02144-2...",39.372206,-76.529453
4,St. John Fisher College,Rochester,NY,14618-3537,"St. John Fisher College, Rochester, NY 0 ...",43.269045,-77.531166
...,...,...,...,...,...,...,...
1932,Putnam History Museum,Cold Spring,NY,10516-2613,"Putnam History Museum, Cold Spring, NY 0 ...",41.42539,-73.944864
1933,Frick Collection,New York,NY,10021-4981,"Frick Collection, New York, NY 0 02144-2...",40.917577,-73.700272
1934,Clark Art Institute,Williamstown,MA,01267-2878,"Clark Art Institute, Williamstown, MA 0 ...",42.74594,-73.163566
1935,Pratt Museum,Homer,AK,99603-7504,"Pratt Museum, Homer, AK 0 02144-2401\n1 ...",59.680543,-151.403995


In [51]:
#Commenting out to avoid accidental runs NEH_List[1] = geocoding(NEH_List[1])
NEH_List[1]

,Organization,Organization City,Organization State,Organization Postal Code,Full Address,Latitude,Longitude
1938,Burke Museum Association,Seattle,WA,98195-3010,"Burke Museum Association, Seattle, WA 0 ...",47.73528,-122.219822
1939,Currier Museum of Art,Manchester,NH,03104-4380,"Currier Museum of Art, Manchester, NH 0 ...",43.051474,-71.375409
1941,Katonah Museum of Art,Katonah,NY,10536-3736,"Katonah Museum of Art, Katonah, NY 0 021...",41.26629,-73.676391
1943,Baranov Museum,Kodiak,AK,99615-6307,"Baranov Museum, Kodiak, AK 0 02144-2401\...",57.813494,-152.355668
1944,Chicago Architecture Foundation,Chicago,IL,60604-2505,"Chicago Architecture Foundation, Chicago, IL 0...",42.023131,-87.523661
...,...,...,...,...,...,...,...
4137,Opera Lafayette,Washington,DC,20003-2141,"Opera Lafayette, Washington, DC 0 02144-...",38.995864,-76.909393
4140,American Philosophical Association,Newark,DE,19716-4200,"American Philosophical Association, Newark, DE...",39.714926,-75.723779
4142,University Press of Colorado,Louisville,CO,80027-1697,"University Press of Colorado, Louisville, CO 0...",39.997869,-105.107848
4145,Montana Department of Commerce,Helena,MT,59601-6282,"Montana Department of Commerce, Helena, MT 0 ...",46.627493,-111.948106


In [56]:
#Concating files back, exporting
NEH_With_Geo = pd.concat(NEH_List)
NEH_With_Geo.to_csv('NEH Org list with Geocoding.csv', index=False)
NEH_With_Geo.drop_duplicates()

,Organization,Organization City,Organization State,Organization Postal Code,Full Address,Latitude,Longitude
0,Tufts University,Somerville,MA,02144-2401,"Tufts University, Somerville, MA 0 02144...",42.418118,-71.072995
1,Northeastern Oklahoma State University,Tahlequah,OK,74464-2301,"Northeastern Oklahoma State University, Tahleq...",35.952788,-94.941901
2,Princeton University,Princeton,NJ,08540-5228,"Princeton University, Princeton, NJ 0 02...",40.391755,-74.617508
3,Goucher College,Baltimore,MD,21204-2753,"Goucher College, Baltimore, MD 0 02144-2...",39.372206,-76.529453
4,St. John Fisher College,Rochester,NY,14618-3537,"St. John Fisher College, Rochester, NY 0 ...",43.269045,-77.531166
...,...,...,...,...,...,...,...
4137,Opera Lafayette,Washington,DC,20003-2141,"Opera Lafayette, Washington, DC 0 02144-...",38.995864,-76.909393
4140,American Philosophical Association,Newark,DE,19716-4200,"American Philosophical Association, Newark, DE...",39.714926,-75.723779
4142,University Press of Colorado,Louisville,CO,80027-1697,"University Press of Colorado, Louisville, CO 0...",39.997869,-105.107848
4145,Montana Department of Commerce,Helena,MT,59601-6282,"Montana Department of Commerce, Helena, MT 0 ...",46.627493,-111.948106


In [57]:
#Running Geocoding and saving files
NEA_List[0] = geocoding(NEA_List[0])
NEA_List[0]

,Organization Name,City,State,Zip,Full Address,Latitude,Longitude
1,1708 Gallery,RICHMOND,VA,23220-4218,"1708 Gallery, RICHMOND, VA 1 23220-4218\...",37.602769,-77.385512
2,18th Street Arts Complex,SANTA MONICA,CA,90404-3807,"18th Street Arts Complex, SANTA MONICA, CA 1 ...",34.05056,-118.443426
3,"369th Experience, Inc.",WASHINGTON,DC,20003-1682,"369th Experience, Inc., WASHINGTON, DC 1 ...",38.995864,-76.909393
4,"3Arts, Inc",CHICAGO,IL,60606-3482,"3Arts, Inc, CHICAGO, IL 1 23220-4218\n2 ...",42.023131,-87.523661
5,500 Sails,"TANAPAG, SAIPAN",MP,96950-8901,"500 Sails, TANAPAG, SAIPAN, MP 1 23220-4...",15.245041,145.761837
...,...,...,...,...,...,...,...
1211,Koahnic Broadcast Corporation,Anchorage,AK,99508-2870,"Koahnic Broadcast Corporation, Anchorage, AK 1...",61.483938,-148.460007
1212,Kokua Kalihi Valley Comprehensive Family Services,HONOLULU,HI,96819-2539,Kokua Kalihi Valley Comprehensive Family Servi...,21.343955,-157.858199
1213,"Kore Press, Inc.",TUCSON,AZ,85705-7656,"Kore Press, Inc., TUCSON, AZ 1 23220-421...",32.320166,-110.708204
1214,Korean Performing Arts Institute of Chicago,CHICAGO,IL,60625-4914,"Korean Performing Arts Institute of Chicago, C...",42.023131,-87.523661


In [58]:
#Running Geocoding and saving files
NEA_List[1] = geocoding(NEA_List[1])
NEA_List[1]

,Organization Name,City,State,Zip,Full Address,Latitude,Longitude
1216,"Ko-Thi, Inc.",MILWAUKEE,WI,53212-1252,"Ko-Thi, Inc., MILWAUKEE, WI 1 23220-4218...",43.194967,-87.863984
1217,"KQED, Inc.",SAN FRANCISCO,CA,94110-1426,"KQED, Inc., SAN FRANCISCO, CA 1 23220-42...",37.929824,-122.28178
1218,Kronos Performing Arts Association,SAN FRANCISCO,CA,94122-2307,"Kronos Performing Arts Association, SAN FRANCI...",37.929824,-122.28178
1219,Kulintang Arts,SAN FRANCISCO,CA,94112-1712,"Kulintang Arts, SAN FRANCISCO, CA 1 2322...",37.929824,-122.28178
1220,"Kundiman, Inc.",NEW YORK,NY,10023-7414,"Kundiman, Inc., NEW YORK, NY 1 23220-421...",40.917577,-73.700272
...,...,...,...,...,...,...,...
2439,Women's Voices Now Inc,REDONDO BEACH,CA,90277-5153,"Women's Voices Now Inc, REDONDO BEACH, CA 1 ...",33.894653,-118.352333
2440,Wonderbound,DENVER,CO,80207-1020,"Wonderbound, DENVER, CO 1 23220-4218\n2 ...",39.914247,-104.600296
2441,Wonderlust Productions,SAINT PAUL,MN,55103-1842,"Wonderlust Productions, SAINT PAUL, MN 1 ...",44.992016,-93.00432
2442,"Woodland Pattern, Inc.",MILWAUKEE,WI,53212-2547,"Woodland Pattern, Inc., MILWAUKEE, WI 1 ...",43.194967,-87.863984


In [59]:
#Running Geocoding and saving files
NEA_List[2] = geocoding(NEA_List[2])
NEA_List[2]

,Organization Name,City,State,Zip,Full Address,Latitude,Longitude
2444,Woolly Mammoth Theatre Company,WASHINGTON,DC,20004-2904,"Woolly Mammoth Theatre Company, WASHINGTON, DC...",38.995864,-76.909393
2445,"Wooster Group, Inc.",NEW YORK,NY,10013-2295,"Wooster Group, Inc., NEW YORK, NY 1 2322...",40.917577,-73.700272
2446,Worcester Art Museum,WORCESTER,MA,01609-3123,"Worcester Art Museum, WORCESTER, MA 1 23...",42.341179,-71.731559
2447,Worcester Center for Performing Arts,WORCESTER,MA,01608-2089,"Worcester Center for Performing Arts, WORCESTE...",42.341179,-71.731559
2448,Worcester Chamber Music Society,WORCESTER,MA,01608-1511,"Worcester Chamber Music Society, WORCESTER, MA...",42.341179,-71.731559
...,...,...,...,...,...,...,...
3103,Virginia Commonwealth University,RICHMOND,VA,23284-9005,"Virginia Commonwealth University, RICHMOND, VA...",37.602769,-77.385512
3104,Virginia Museum of Fine Arts,RICHMOND,VA,23220-4007,"Virginia Museum of Fine Arts, RICHMOND, VA 1 ...",37.602769,-77.385512
3106,Virginia Symphony Orchestra,NORFOLK,VA,23510-1626,"Virginia Symphony Orchestra, NORFOLK, VA 1 ...",37.040886,-76.133872
3107,Virginia's Heritage Music Trail The Crooked Road,ABINGDON,VA,24210,Virginia's Heritage Music Trail The Crooked Ro...,36.732008,-81.917944


In [60]:
#Running Geocoding and saving files
NEA_List[3] = geocoding(NEA_List[3])
NEA_List[3]

,Organization Name,City,State,Zip,Full Address,Latitude,Longitude
3113,"Visual Arts Center of New Jersey, a New Jersey...",SUMMIT,NJ,07901-3472,"Visual Arts Center of New Jersey, a New Jersey...",40.739229,-74.330619
3115,"Visual Studies Workshop, Inc.",ROCHESTER,NY,14607-1405,"Visual Studies Workshop, Inc., ROCHESTER, NY 1...",43.269045,-77.531166
3118,"Vivian Beaumont Theater, Inc.",NEW YORK,NY,10023-6916,"Vivian Beaumont Theater, Inc., NEW YORK, NY 1 ...",40.917577,-73.700272
3125,Vox Femina Los Angeles,CULVER CITY,CA,90232-7429,"Vox Femina Los Angeles, CULVER CITY, CA 1 ...",34.03508,-118.369577
3130,"Ward Foundation, Inc.",SALISBURY,MD,21804-8722,"Ward Foundation, Inc., SALISBURY, MD 1 2...",38.427743,-75.537306
...,...,...,...,...,...,...,...
661,Coker University,HARTSVILLE,SC,29550-3742,"Coker University, HARTSVILLE, SC 1 23220...",34.394997,-80.043762
675,Comfort Station NFP,CHICAGO,IL,60625-9267,"Comfort Station NFP, CHICAGO, IL 1 23220...",42.023131,-87.523661
677,ComMotion - Community in Motion,RALEIGH,NC,27617-7206,"ComMotion - Community in Motion, RALEIGH, NC 1...",35.971728,-78.471063
681,Community Foundation of Grand Forks East Grand...,Grand Forks,ND,58201-4531,Community Foundation of Grand Forks East Grand...,47.969701,-97.009802


In [61]:
#Running Geocoding and saving files
NEA_List[4] = geocoding(NEA_List[4])
NEA_List[4]

,Organization Name,City,State,Zip,Full Address,Latitude,Longitude
686,Community Television Network,CHICAGO,IL,60647-4328,"Community Television Network, CHICAGO, IL 1 ...",42.023131,-87.523661
689,Community Theatre of Greensboro,GREENSBORO,NC,27406-1326,"Community Theatre of Greensboro, GREENSBORO, N...",36.215324,-79.659866
692,Comunidad y Herencia Cultural,SPRINGFIELD,OR,97477-7550,"Comunidad y Herencia Cultural, SPRINGFIELD, OR...",44.095459,-122.87973
694,Concerto Soloists,Philadelphia,PA,19102-4400,"Concerto Soloists, Philadelphia, PA 1 23...",40.137992,-74.955763
695,"Concerts in Motion, Inc.",NEW YORK,NY,10019-6896,"Concerts in Motion, Inc., NEW YORK, NY 1 ...",40.917577,-73.700272
...,...,...,...,...,...,...,...
3936,Youth in Focus,SEATTLE,WA,98144-4645,"Youth in Focus, SEATTLE, WA 1 23220-4218...",47.73528,-122.219822
3937,"Youth Justice Network, Inc.",NEW YORK,NY,10027-4590,"Youth Justice Network, Inc., NEW YORK, NY 1 ...",40.917577,-73.700272
3941,Youth Spirit Artworks Inc.,BERKELEY,CA,94703-2713,"Youth Spirit Artworks Inc., BERKELEY, CA 1 ...",37.906689,-122.234196
3954,"Zorongo Flamenco, Inc.",MINNEAPOLIS,MN,55406-2630,"Zorongo Flamenco, Inc., MINNEAPOLIS, MN 1 ...",45.05125,-93.193794


In [62]:
#Concating files back, exporting
NEA_With_Geo = pd.concat(NEA_List)
NEA_With_Geo.to_csv('NEA_Org list with Geocoding.csv', index=False)
NEA_With_Geo.drop_duplicates()

,Organization Name,City,State,Zip,Full Address,Latitude,Longitude
1,1708 Gallery,RICHMOND,VA,23220-4218,"1708 Gallery, RICHMOND, VA 1 23220-4218\...",37.602769,-77.385512
2,18th Street Arts Complex,SANTA MONICA,CA,90404-3807,"18th Street Arts Complex, SANTA MONICA, CA 1 ...",34.05056,-118.443426
3,"369th Experience, Inc.",WASHINGTON,DC,20003-1682,"369th Experience, Inc., WASHINGTON, DC 1 ...",38.995864,-76.909393
4,"3Arts, Inc",CHICAGO,IL,60606-3482,"3Arts, Inc, CHICAGO, IL 1 23220-4218\n2 ...",42.023131,-87.523661
5,500 Sails,"TANAPAG, SAIPAN",MP,96950-8901,"500 Sails, TANAPAG, SAIPAN, MP 1 23220-4...",15.245041,145.761837
...,...,...,...,...,...,...,...
3936,Youth in Focus,SEATTLE,WA,98144-4645,"Youth in Focus, SEATTLE, WA 1 23220-4218...",47.73528,-122.219822
3937,"Youth Justice Network, Inc.",NEW YORK,NY,10027-4590,"Youth Justice Network, Inc., NEW YORK, NY 1 ...",40.917577,-73.700272
3941,Youth Spirit Artworks Inc.,BERKELEY,CA,94703-2713,"Youth Spirit Artworks Inc., BERKELEY, CA 1 ...",37.906689,-122.234196
3954,"Zorongo Flamenco, Inc.",MINNEAPOLIS,MN,55406-2630,"Zorongo Flamenco, Inc., MINNEAPOLIS, MN 1 ...",45.05125,-93.193794
